# Training Deep Neural Networks

We will explore the vanishing and exploding gradients problems, tackle complex tasks when you have little labeled data, various optimizers to speed up training models and a few popular regularization techniques.

## Setup

In [45]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


![Vanishing-Gradients-in-DNN.png](images/Vanishing-Gradients-in-DNN.png)

### Glorot and He Initialization

In their paper, Glorot and Bengio proposed that for the signal to flow properly, variance of the outputs of each layer must be equal to variance of the input; and we need gradience to have equal variance before and after flowing through a layer in the reverse direction. 

Glorot initialization (when using logistic activation function) :

Normal distribution with mean 0 and variance $\sigma^2 = \frac{1}{fan_{avg}}$

Or a uniform distribution between -r and +r, with r = $\sqrt(\frac{3}{fan_{avg}})$

Here, fan$_{avg}$ = (fan$_{in}$ + fan$_{out}$) / 2

This initialization strategy is called **Glorot Initialization**.

Yann LeCun proposed replacing fan$_{avg}$ with fan$_{in}$ in above equation. This strategy is called **LeCun initialization**.

The initialization strategy for the *ReLU* activation functions (including variants) is sometimes called the **He initialization**.

*Table: Initialization parameters for each type of activation function:*

| Initialization | Activation functions | $\sigma^2(Normal)$ |
|-|-|-|
| Glorot | None, tanh, logistic, softmax | $\frac{1}{fan_{avg}}$ |
|He|ReLU and variants| $\frac{2}{fan_{in}}$ |
|LeCun|SELU| $\frac{1}{fan_{in}}$ |

In [44]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

By default, Keras uses Glorot initialization with a uniform distribution.

In [3]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

If you want He initialization with a uniform distribution but based on fan$_{avg}$ rather than fan$_{in}$, you can use VarianceScaling initializer like this:

In [4]:
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

### Nonsaturating Activation functions

**ReLU** is considered as the much better activation function compared to others. Unfortunately, it is not perfect. It suffers from a problem known as *dying ReLUs*: during training, some neurons stop outputting anything other than 0, especially if you use a large learning rate.

To solve this problem you may use a variant of ReLU, such as the **Leaky ReLU**. This function is defined as: LeakyReLU$_\alpha$(z) = max($\alpha$z, z). $\alpha$ is a slope of the function for z < 0 and is typically set to 0.01. This small slope ensures that the ReLU will never "die"; they can go into a long coma, but eventually wake up.

In the *randomized leaky ReLU* (RReLU), the $\alpha$ is picked randomly in a given range during training and is fixed to an averarage during testing. It acts as a regularizer (reduces the risk of overfitting).

In the *parametric leaky ReLU* (PReLU), the $\alpha$ is authorized to be learned during training (instead of being a hyperparameter, it becomes a parameter that can be modified during backpropagation). PReLU is reported to strongly outperform ReLU on large image datasets, but on smaller datasets it runs the risk of overfitting.

![Leaky ReLU](images/leaky_relu.png)
<center>Leaky ReLU; like ReLU, but with small slope for -ve values.</center>

In [5]:
[m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'swish',
 'tanh']

In [6]:
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [11]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_val, X_train = X_train_full[:5000], X_train_full[5000:]
y_val, y_train = y_train_full[:5000], y_train_full[5000:]

In [12]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [13]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [14]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7902
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5553 - accuracy: 0.8157 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5338 - accuracy: 0.8225 - val_loss: 0.5157 - val_accuracy:

Now let's try PReLU:

In [15]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [16]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [18]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7117 - accuracy: 0.7648 - val_loss: 0.6616 - val_accuracy: 0.7836
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6421 - accuracy: 0.7888 - val_loss: 0.6070 - val_accuracy: 0.8036
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5976 - accuracy: 0.8030 - val_loss: 0.5795 - val_accuracy: 0.8116
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5661 - accuracy: 0.8134 - val_loss: 0.5442 - val_accuracy: 0.8216
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5427 - accuracy: 0.8198 - val_loss: 0.5234 - val_accuracy: 0.8310
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5240 - accuracy: 0.8244 - val_loss: 0.5077 - val_accuracy: 0.8330
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5088 - accuracy: 0.8288 - val_loss: 0.4932 - val_accuracy:


**Exponential linear unit (ELU)** outperforms all ReLU variants: training time is reduced and the network performed better on the test set.

ELU activation function: 

 $$ ELU_\alpha(z) =   \left\{
\begin{array}{ll}
      \alpha(exp(z) -1) & if z < 0 \\
      z & if z >= 0 \\
\end{array} 
\right.  $$

![ELU](images/ELU.png)


The main drawback of ELU is it is slower to compute than the ReLU functions (due to exponential function).

Implementing ELU in TensorFlow is trivial, just specify the activation function when building each layer:

In [19]:
keras.layers.Dense(10, activation="elu")

Scaled ELU (**SELU**) activation function:

During training, a neural network composed exclusively of a stack of dense layers using the SELU activation function and LeCun initialization will self-normalize: the output of each layer will tend to preserve the same mean and variance during training, which solves the vanishing/exploding gradients problem.

As a result, this activation function outperforms the other activation functions very significantly for such neural nets. Unfortunately, the self-normalizing property of the SELU activation function is easily broken: you cannot use ℓ1 or ℓ2 regularization, regular dropout, max-norm, skip connections or other non-sequential topologies (so recurrent neural networks won't self-normalize). 

However, in practice it works quite well with sequential CNNs. If you break self-normalization, SELU will not necessarily outperform other activation functions.

![SELU](images/selu.png)

Let's create a neural net for Fashion MNIST with 100 hidden layers, using the SELU activation function:

In [26]:
np.random.seed(42)
tf.random.set_seed(42)

In [27]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [28]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

**Note**: Do not forget to scale the inputs to mean 0 and standard deviation 1:

In [36]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_val_scaled = (X_val - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [37]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_val_scaled, y_val))

Epoch 1/5
1719/1719 [==============================] - 23s 13ms/step - loss: 0.9661 - accuracy: 0.6431 - val_loss: 0.6958 - val_accuracy: 0.7580
Epoch 2/5
1719/1719 [==============================] - 21s 12ms/step - loss: 0.6734 - accuracy: 0.7562 - val_loss: 0.5860 - val_accuracy: 0.7894
Epoch 3/5
1719/1719 [==============================] - 23s 13ms/step - loss: 0.6586 - accuracy: 0.7634 - val_loss: 0.7729 - val_accuracy: 0.6948
Epoch 4/5
1719/1719 [==============================] - 23s 14ms/step - loss: 0.5943 - accuracy: 0.7900 - val_loss: 0.5439 - val_accuracy: 0.8112
Epoch 5/5
1719/1719 [==============================] - 20s 12ms/step - loss: 0.5279 - accuracy: 0.8157 - val_loss: 0.5446 - val_accuracy: 0.8140


In [43]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_val_scaled, y_val))

Epoch 1/5
1719/1719 [==============================] - 22s 13ms/step - loss: 1.7682 - accuracy: 0.2741 - val_loss: 1.3060 - val_accuracy: 0.3864
Epoch 2/5
1719/1719 [==============================] - 21s 12ms/step - loss: 1.2256 - accuracy: 0.4781 - val_loss: 1.1808 - val_accuracy: 0.4822
Epoch 3/5
1719/1719 [==============================] - 20s 12ms/step - loss: 0.9104 - accuracy: 0.6321 - val_loss: 0.8720 - val_accuracy: 0.6402
Epoch 4/5
1719/1719 [==============================] - 19s 11ms/step - loss: 0.7976 - accuracy: 0.6908 - val_loss: 0.7200 - val_accuracy: 0.7324
Epoch 5/5
1719/1719 [==============================] - 19s 11ms/step - loss: 0.7394 - accuracy: 0.7217 - val_loss: 0.7023 - val_accuracy: 0.7286


Not great at all, we suffered from the vanishing/exploding gradients problem.

#### So which activation function should you use?

* In general, SELU > ELU > leaky ReLU (all variants) > ReLU > tanh > logistic.
* If the network's architecture prevents it from self-normalizing, ELU may perform better than SELU.
* For runtime latency, prefer Leaky ReLU.
* If you have spare time and computing power, you can use cross-validation to evaluate RReLU(if overfitting) or PReLU (for huge training set).
* If speed is priority, ReLU is the best choice.

### Batch Normalization

In [46]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [47]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_416 (Dense)            (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_417 (Dense)            (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_418 (Dense)            (None, 10)               

In [48]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [50]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [51]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8750 - accuracy: 0.7122 - val_loss: 0.5524 - val_accuracy: 0.8226
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5753 - accuracy: 0.8032 - val_loss: 0.4725 - val_accuracy: 0.8468
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5189 - accuracy: 0.8206 - val_loss: 0.4375 - val_accuracy: 0.8548
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4827 - accuracy: 0.8324 - val_loss: 0.4152 - val_accuracy: 0.8600
Epoch 5/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4565 - accuracy: 0.8408 - val_loss: 0.3997 - val_accuracy: 0.8634
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4398 - accuracy: 0.8474 - val_loss: 0.3866 - val_accuracy: 0.8694
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4242 - accuracy: 0.8514 - val_loss: 0.3762 - val_accuracy:

### Gradient Clipping